In [22]:
import re
from kss import Kss

from pathlib import Path
import json

path = Path("./quotes_samples/table_recipe.json")
with open(path, "r") as f:
    data = json.load(f)

sentence_split = Kss("split_sentences")
answer = data["answer"]

In [6]:
import pycmarkgfm
from pycmarkgfm import options
html = pycmarkgfm.gfm_to_html(answer, options=options.sourcepos)

In [7]:
html

'<p data-sourcepos="1:1-1:82">김치찌개 레시피에 필요한 재료를 표로 정리해 드리겠습니다.</p>\n<table data-sourcepos="3:1-22:123">\n<thead>\n<tr data-sourcepos="3:1-3:16">\n<th data-sourcepos="3:2-3:9">재료</th>\n<th data-sourcepos="3:11-3:15">양</th>\n</tr>\n</thead>\n<tbody>\n<tr data-sourcepos="5:1-5:160">\n<td data-sourcepos="5:2-5:9">김치</td>\n<td data-sourcepos="5:11-5:159">1쪽<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 280g(1/4 포기)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->11<!-- raw HTML omitted --></td>\n</tr>\n<tr data-sourcepos="6:1-6:284">\n<td data-sourcepos="6:2-6:15">돼지고기</td>\n<td data-sourcepos="6:17-6:283">200g(앞다리 살)<!-- raw HTML omitted -->1<!-- raw HTML omitted -->, 250g<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 150g(목등심)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 200g<!-- raw HTML omitted -->8<!-- raw HTML omitted -->, 200g(또는 소고기)<!-- raw HTML omitted -->9<!-- raw HTML omitted

In [8]:
from IPython.display import HTML
HTML(html)

재료,양
김치,"1쪽<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 280g(1/4 포기)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
돼지고기,"200g(앞다리 살)<!-- raw HTML omitted -->1<!-- raw HTML omitted -->, 250g<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 150g(목등심)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 200g<!-- raw HTML omitted -->8<!-- raw HTML omitted -->, 200g(또는 소고기)<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 150g(혹은 소고기)<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
두부,"반 모<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1/2모<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
대파,"약간<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1대<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 1대<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
된장,"1스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->9<!-- raw HTML omitted -->"
고춧가루,"1스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1작은술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
설탕,"반 스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 0.5스푼<!-- raw HTML omitted -->8<!-- raw HTML omitted -->"
새우젓,약간<!-- raw HTML omitted -->2<!-- raw HTML omitted -->
다진 마늘,"1큰술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->8<!-- raw HTML omitted -->"
생강즙,1큰술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->


In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [14]:
soup.p["data-sourcepos"]

'1:1-1:82'

In [24]:
print(answer)
print((4+4+3+3+2+3+5)*3+6+1)

김치찌개 레시피에 필요한 재료를 표로 정리해 드리겠습니다.

| 재료 | 양 |
| --- | --- |
| 김치 | 1쪽<span class='style'>2</span>, 280g(1/4 포기)<span class='style'>6</span>, 2컵<span class='style'>9</span>, 2컵<span class='style'>11</span> |
| 돼지고기 | 200g(앞다리 살)<span class='style'>1</span>, 250g<span class='style'>2</span>, 150g(목등심)<span class='style'>6</span>, 200g<span class='style'>8</span>, 200g(또는 소고기)<span class='style'>9</span>, 150g(혹은 소고기)<span class='style'>11</span> |
| 두부 | 반 모<span class='style'>2</span>, 1/2모<span class='style'>11</span> |
| 대파 | 약간<span class='style'>2</span>, 1대<span class='style'>9</span>, 1대<span class='style'>11</span> |
| 된장 | 1스푼<span class='style'>2</span>, 1큰술<span class='style'>9</span> |
| 고춧가루 | 1스푼<span class='style'>2</span>, 1작은술<span class='style'>6</span>, 1큰술<span class='style'>11</span> |
| 설탕 | 반 스푼<span class='style'>2</span>, 0.5스푼<span class='style'>8</span> |
| 새우젓 | 약간<span class='style'>2</span> |
| 다진 마늘 | 1큰술<span class='style'>6</span>, 1큰술<span class='style'>8

In [4]:
import pycmarkgfm
from pycmarkgfm import options
html = pycmarkgfm.gfm_to_html("글자수 판별123", options=options.sourcepos)
print(html)

<p data-sourcepos="1:1-1:19">글자수 판별123</p>



In [7]:
html = pycmarkgfm.gfm_to_html("12345->%^&", options=options.sourcepos)
print(html)

<p data-sourcepos="1:1-1:10">12345-&gt;%^&amp;</p>



In [20]:
import pycmarkgfm
from bs4 import BeautifulSoup

def create_byte_to_char_map(markdown_string):
    byte_to_char = {}
    char_pos = 0
    for byte_pos, byte in enumerate(markdown_string.encode('utf-8')):
        byte_to_char[byte_pos] = char_pos
        if byte >> 6 != 0b10:  # This is the start of a new UTF-8 character
            char_pos += 1
    return byte_to_char

def sourcepos_to_string_indices(markdown_string, sourcepos):
    byte_to_char = create_byte_to_char_map(markdown_string)
    
    start, end = sourcepos.split('-')
    start_line, start_col = map(int, start.split(':'))
    end_line, end_col = map(int, end.split(':'))
    
    lines = markdown_string.split('\n')
    
    start_byte = sum(len(line.encode('utf-8')) + 1 for line in lines[:start_line-1])
    start_byte += len(lines[start_line-1][:start_col-1].encode('utf-8'))
    
    end_byte = sum(len(line.encode('utf-8')) + 1 for line in lines[:end_line-1])
    end_byte += len(lines[end_line-1][:end_col-1].encode('utf-8'))
    
    return byte_to_char[start_byte], byte_to_char[end_byte]

# Example usage
markdown = "Hello 안녕하세요 World\n\nThis is a new line"
html = pycmarkgfm.gfm_to_html(markdown, options=options.sourcepos|options.smart)

soup = BeautifulSoup(html)

# Let's say we extracted this sourcepos from an HTML element
sourcepos = soup.p["data-sourcepos"]  # This should correspond to "안녕하세요" in the first line

start_index, end_index = sourcepos_to_string_indices(markdown, sourcepos)
print(f"The text from index {start_index} to {end_index} is: {markdown[start_index:end_index]}")

The text from index 0 to 17 is: Hello 안녕하세요 World


In [17]:
html

'<p data-sourcepos="1:1-1:27">Hello 안녕하세요 World</p>\n<p data-sourcepos="3:1-3:18">This is a new line</p>\n'

In [53]:
path = Path("./quotes_samples/bullet_rag.json")
with open(path, "r") as f:
    data = json.load(f)

sentence_split = Kss("split_sentences")
answer = data["answer"]

# Example usage
markdown = answer
html = pycmarkgfm.gfm_to_html(markdown, options=options.sourcepos|options.smart)

soup = BeautifulSoup(html)

# Let's say we extracted this sourcepos from an HTML element
sourcepos = soup.find_all("p")[1]["data-sourcepos"]

start_index, end_index = sourcepos_to_string_indices(markdown, sourcepos)
print(f"The text from index {start_index} to {end_index} is: {markdown[start_index:end_index]}")

KeyError: 1333

In [54]:
soup.find_all("p")[1]

<p data-sourcepos="12:1-12:77">RAG에 대해 더 궁금한 점이 있으시면 언제든지 물어보세요.</p>

In [51]:
from IPython.display import HTML
HTML(html)

재료,양
김치,"1쪽<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 280g(1/4 포기)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 2컵<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
돼지고기,"200g(앞다리 살)<!-- raw HTML omitted -->1<!-- raw HTML omitted -->, 250g<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 150g(목등심)<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 200g<!-- raw HTML omitted -->8<!-- raw HTML omitted -->, 200g(또는 소고기)<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 150g(혹은 소고기)<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
두부,"반 모<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1/2모<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
대파,"약간<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1대<!-- raw HTML omitted -->9<!-- raw HTML omitted -->, 1대<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
된장,"1스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->9<!-- raw HTML omitted -->"
고춧가루,"1스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 1작은술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->11<!-- raw HTML omitted -->"
설탕,"반 스푼<!-- raw HTML omitted -->2<!-- raw HTML omitted -->, 0.5스푼<!-- raw HTML omitted -->8<!-- raw HTML omitted -->"
새우젓,약간<!-- raw HTML omitted -->2<!-- raw HTML omitted -->
다진 마늘,"1큰술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->, 1큰술<!-- raw HTML omitted -->8<!-- raw HTML omitted -->"
생강즙,1큰술<!-- raw HTML omitted -->6<!-- raw HTML omitted -->


In [52]:
print(answer)

김치찌개 레시피에 필요한 재료를 표로 정리해 드리겠습니다.

| 재료 | 양 |
| --- | --- |
| 김치 | 1쪽<span class='quote' style='font-weight: bold'>2</span>, 280g(1/4 포기)<span class='quote' style='font-weight: bold'>6</span>, 2컵<span class='quote' style='font-weight: bold'>9</span>, 2컵<span class='quote' style='font-weight: bold'>11</span> |
| 돼지고기 | 200g(앞다리 살)<span class='quote' style='font-weight: bold'>1</span>, 250g<span class='quote' style='font-weight: bold'>2</span>, 150g(목등심)<span class='quote' style='font-weight: bold'>6</span>, 200g<span class='quote' style='font-weight: bold'>8</span>, 200g(또는 소고기)<span class='quote' style='font-weight: bold'>9</span>, 150g(혹은 소고기)<span class='quote' style='font-weight: bold'>11</span> |
| 두부 | 반 모<span class='quote' style='font-weight: bold'>2</span>, 1/2모<span class='quote' style='font-weight: bold'>11</span> |
| 대파 | 약간<span class='quote' style='font-weight: bold'>2</span>, 1대<span class='quote' style='font-weight: bold'>9</span>, 1대<span class='quote' style='font-weight